In [25]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
# Add parent dir to path, so that python finds the lenskit package
sys.path.insert(0,parentdir) 

In [26]:
from lenskit.metrics import dataGenerator
from lenskit import batch, topn, util, topnFair
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als, funksvd, user_knn as uknn, item_knn as iknn 
from lenskit import topn, topnFair

In [27]:
import numpy as np
import pandas as pd
import math
import pickle
%matplotlib inline

In [32]:
#dataset = os.path.join(parentdir,'ml-latest-small/ratings.1.csv')    
#ratings = pd.read_csv(dataset, sep=',',
#                      names=['user', 'item', 'rating', 'timestamp'], header=0)


dataset = os.path.join(parentdir,'movielens100k/u.data')     
ratings = pd.read_csv(dataset, sep='\s+',
                      names=['user', 'item', 'rating', 'timestamp'], header=0)

items = pd.read_csv('/Users/josse/Git-kode projects/Speciale/Fair-Recommendations/ml-latest-small/movies.csv', sep=',',
                      names=['item','title','genres'], header=0)

In [33]:
items_dummy = pd.concat([items, items['genres'].str.get_dummies(sep='|')], axis=1)
items_dummy.head()

,item,title,genres,(no genres listed),Action,Adventure,Animation,Children,Comedy,Crime,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,2,Jumanji (1995),Adventure|Children|Fantasy,0,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,Grumpier Old Men (1995),Comedy|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,5,Father of the Bride Part II (1995),Comedy,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
ratings.shape

(99999, 4)

In [37]:
ratings1 = ratings.join(items_dummy.set_index('item'), on='item')
ratings1 = ratings1.dropna().reset_index()
ratings1.shape


(84205, 27)

In [38]:
algo_uu = uknn.UserUser(20)
algo_ii = iknn.ItemItem(20)
algo_als = als.BiasedMF(50)
algo_funkSVD = funksvd.FunkSVD(50)


In [39]:
def eval(aname, algo, train, test):
    fittable = util.clone(algo)
    fittable = Recommender.adapt(fittable)
    fittable.fit(train)
    users = test.user.unique()
    
    # now we run the recommender
    recs = batch.recommend(fittable, users, items_n)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

In [ ]:
all_recs = []
test_data = []
for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 5, xf.SampleFrac(0.2)):
    test_data.append(test)
    all_recs.append(eval('ItemItem', algo_ii, train, test))
    all_recs.append(eval('UserUser', algo_uu, train, test))
    all_recs.append(eval('ALS', algo_als, train, test))
    all_recs.append(eval('FunkSVD', algo_funkSVD, train, test))
    

In [10]:
test_data = pd.concat(test_data, ignore_index=True)
all_recs = pd.concat(all_recs, ignore_index=True)
all_recs_joined = all_recs.join(items_dummy.set_index('item'), on='item')
all_recs_joined.head()

,item,score,user,rank,Algorithm,title,genres,(no genres listed),Action,Adventure,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,592,4.382521,5,1,ItemItem,Batman (1989),Action|Crime|Thriller,0,1,0,...,0,0,0,0,0,0,0,1,0,0
1,480,4.210691,5,2,ItemItem,Jurassic Park (1993),Action|Adventure|Sci-Fi|Thriller,0,1,1,...,0,0,0,0,0,0,1,1,0,0
2,349,4.210691,5,3,ItemItem,Clear and Present Danger (1994),Action|Crime|Drama|Thriller,0,1,0,...,0,0,0,0,0,0,0,1,0,0
3,296,4.210691,5,4,ItemItem,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,356,4.148607,5,5,ItemItem,Forrest Gump (1994),Comedy|Drama|Romance|War,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [11]:
test_data.to_pickle("test_data_Lenskit1")
all_recs_joined.to_pickle("all_recs_joined_Lenskit1")

In [12]:
#test = all_recs_joined.loc[all_recs_joined["user"]==1]
#test = test.loc[test["Algorithm"] == "ALS"]
#test.reset_index()
#test.iloc[90:100,:]

In [13]:
#providers = list(all_recs_joined.iloc[:,8:])
#items_n = all_recs_joined.item.nunique()
#proItems_n =  all_recs_joined.loc[ all_recs_joined['Drama'] == 1].item.nunique()
#proItems_n_horror =  all_recs_joined.loc[ all_recs_joined['Horror'] == 1].item.nunique()

In [14]:
#rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
#rla.add_metric("rND")
#rla.add_metric("rKL")
#rla.add_metric("rRD")
#rla.add_metric("div")
#rla.add_metric("dem_parity")
#rla.add_metric("ndcg")
#results = rla.compute(all_recs_joined, test_data, "Drama", providers, items_n, proItems_n)
#results.head()

In [15]:
#results.groupby('Algorithm').mean()

In [16]:
#rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
#rla.add_metric("rND")
#rla.add_metric("rKL")
#rla.add_metric("rRD")
#rla.add_metric("div")
#rla.add_metric("dem_parity")
#rla.add_metric("ndcg")
#results_horror = rla.compute(all_recs_joined, test_data, "Horror", providers, items_n, proItems_n_horror)
#results_horror.groupby('Algorithm').mean()

In [17]:
#allResults = []
#allResults.append(results.groupby('Algorithm').mean())
#allResults.append(results_horror.groupby('Algorithm').mean())

In [18]:
#pd.concat(allResults, keys=['Drama', 'Horror'])

In [19]:
def evaluate_genre(genre): 
    
    providers = list(all_recs_joined.iloc[:,8:])
    proItems_genre_n =  all_recs_joined.loc[ all_recs_joined[genre] == 1].item.nunique()
    print (genre)
    rla = topnFair.FairRecListAnalysis(['user', "Algorithm"])
    rla.add_metric("rND")
    rla.add_metric("rKL")
    rla.add_metric("rRD")
    rla.add_metric("APCR")
    rla.add_metric("nd_APCR")
    rla.add_metric("equal_ex")
    rla.add_metric("ndcg")
    results= rla.compute(all_recs_joined, test_data, genre, providers)
    #results_horror.groupby('Algorithm').mean()
    print("done" , genre)
    return results.groupby('Algorithm').mean()

In [20]:
testproviders = ["Drama", "Action"]
#testproviders = list(all_recs_joined.iloc[:,8:])
allResults = []
for genre in testproviders:
    rep = evaluate_genre(genre)
    allResults.append(rep)    

Drama


C:\Users\josse\Anaconda3\lib\site-packages\pandas\core\indexing.py:1472: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


done Drama
Action
done Action


In [21]:
output = pd.concat(allResults, keys=testproviders)
output

rND       rKL       rRD      APCR   nd_APCR  equal_ex  \
       Algorithm                                                               
Drama  ALS        0.151771  0.033607  0.212734  0.989474  0.885840  0.990347   
       FunkSVD    0.110977  0.019564  0.066541  0.989474  0.917805  1.014908   
       ItemItem   0.327558  0.083060  0.000000  0.768421  0.616374  1.259245   
       UserUser   0.157946  0.036064  0.099462  0.934211  0.842073  1.061508   
Action ALS        0.106706  0.027449  0.129612  0.989474  0.885840  0.973797   
       FunkSVD    0.082331  0.016608  0.146743  0.989474  0.917805  1.003600   
       ItemItem   0.281254  0.100661  0.129794  0.768421  0.616374  1.073484   
       UserUser   0.116975  0.028610  0.234712  0.934211  0.842073  0.911780   

                      ndcg  
       Algorithm            
Drama  ALS        0.114063  
       FunkSVD    0.093702  
       ItemItem   0.034385  
       UserUser   0.054316  
Action ALS        0.114063  
       FunkSVD    0.093702  
       ItemItem   0.034385  
       UserUser   0.054316